# Form Cropping

#### This process aims to crop and transform the forms into proper size and orientation for field detections

In [1]:
from __future__ import print_function
from imutils import perspective
from imutils import contours
from matplotlib import pyplot as plt
from PIL import Image
from skimage import io , img_as_float
from scipy.spatial import distance as dist
from sklearn.cluster import KMeans
from scipy.signal import convolve2d


import argparse
import cv2
import numpy as np
import pandas as pd
import image_slicer
import imutils
import math
import os

In [2]:
class cPoints:
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.P = -1
    def assignP(self, Pin):
        self.P = Pin
        

### Corner detection algorithsm

In [3]:
def cornerDetect(Img):
    img = cv2.imread(Img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    corners = cv2.goodFeaturesToTrack(gray,maxCorners = 4,qualityLevel=0.01,minDistance = 10,useHarrisDetector=True)
    corners = np.int0(corners)
    #print(corners)
    for i in corners:
        x,y = i.ravel()
        cv2.circle(img,(x,y),3,255,-1)
    cv2.imwrite('GFeature.png',img)
    #plt.imshow(img),plt.show()
    return corners

### Noise estmation

In [4]:
def estimate_noise(I):
    I = cv2.cvtColor(I,cv2.COLOR_BGR2GRAY);

    H,W = I.shape
   
    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]
    
    sigma = np.sum(np.sum(np.absolute(convolve2d(I, M))))
    sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))

    return sigma

### Line detection Function

In [5]:
def lineDetectFun(img):
    img = cv2.imread(img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    minLineLength = 5
    maxLineGap = 70
    lines = cv2.HoughLinesP(edges,rho=1,theta=np.pi/180,threshold=100,minLineLength=minLineLength,maxLineGap=maxLineGap)
    
    for line in lines:
        #print(line)
        for x1,y1,x2,y2 in line:
            #need to process each datapoint here find the important lines with key values in every coordinates
            cv2.line(img = img,pt1 = (x1,y1),pt2 = (x2,y2), color = (0,255,0), thickness = 1)

    cv2.imwrite('houghlines5.png',img)
    
    #return img

In [6]:
def convertFromListToArray(listO):
    dummyList = []
    for i in listO:
        dummyList.append(i[0])
    
    return np.array([np.array(xi) for xi in dummyList])

### Loop over each contour for size

In [7]:
def findContours(img):
    
    image = cv2.imread(img)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY);
    #find major contours in the image
    cnts,_ = cv2.findContours(image.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the convex hull object for each contour
    hull_list = []
    for i in range(len(cnts)):
        hull = cv2.convexHull(cnts[i])
        hull_list.append(hull)
    
    maxArea = -1
    valid_Draw = []
    for c in hull_list:
        if cv2.contourArea(c)<maxArea:
            continue
        maxArea = cv2.contourArea(c)
        valid_cnts = c
        valid_Draw.clear()
        valid_Draw.append(c)
    
    # Draw contours + hull results
    for i in range(len(valid_Draw)):
        cv2.drawContours(image, valid_Draw, i, (255, 153, 255))
        
        
    cv2.imwrite("resultContour.jpg", image)
    Result = convertFromListToArray(valid_cnts)
    
    
    image = cv2.fillPoly(image, pts = [np.array(valid_cnts)], color=(255,255,255))
    stencil = np.zeros(image.shape).astype(image.dtype)

    color = [255, 255, 255]
    cv2.fillPoly(stencil,[np.array(valid_cnts)] , color)
    resultimage = cv2.bitwise_and(image, stencil)
    cv2.imwrite("result.jpg", resultimage)
    
    #convert the points to the right format
    
   
    # draw the contour and show it
    
    return Result

### Edge detection Function

In [8]:
def edgeDetectFun(Images):

#     # find the average pixel value for the image

    img = cv2.imread(Images)
    
    #estimate noise for the image
    
    
    sigma = estimate_noise(img)
    print(sigma)
    if sigma >4 :
        img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, None)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, None)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, None)

        #img = cv2.dilate(img, None, iterations=1)
        img = cv2.erode(img, None, iterations=1)

        img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.adaptiveThreshold(img_grey,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,7,3)
    
    cv2.imwrite('bwimage.png',img)

    edges = cv2.Canny(img,10,20)
    #edges = cv2.dilate(edges, None, iterations=1)
    #edges = cv2.erode(edges, None, iterations=1)
    
    cv2.imwrite('edgeTest.png',edges)

    img = lineDetectFun('edgeTest.png')

    contourCoordinates = findContours('houghlines5.png')

    
#     plt.subplot(121),plt.imshow(img,cmap = 'gray')
#     plt.title('Original Image'), plt.xticks([]), plt.yticks([])
#     plt.subplot(122),plt.imshow(edges,cmap = 'gray')
#     plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
   

#     plt.rcParams['figure.figsize'] = [200, 50]
#     fig=plt.figure(figsize=(180, 160))
  
    
    return contourCoordinates
    

## Processing Function

In [9]:
def cornerIdentify(points):
     # find the corresponding corners for the elements
    points = sorted(points,key=sum)
    #print(points)

    tl = points[0]
    br = points[3]

    if points[1][0]>points[2][0]:
        tr = points[1]
        bl = points[2]
    elif points[1][0]<points[2][0]:
        tr = points[2]
        bl = points[1]
    elif points[1][1] > points[2][1] :
        # if for whatever reason, x value is the same, which is highly unlikely
        #compare y value
        tr = points[2]
        br = points[1]
    else:
        tr = points[1]
        br = points[2]
    return tl, tr, bl, br

In [10]:
def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

In [11]:
def adjustmentfunc(tl,tr,bl,br):
    top = [tr[0]-tl[0],tr[1]-tl[1]]
    right = [br[0]-tr[0],br[1]-tr[1]]
    bottom = [bl[0]-br[0], bl[1]-br[1]]
    left = [tl[0]-bl[0], tl[1]-bl[1]]
    
    trAngle = angle(left, top)
    tlAngle = angle(top,right)
    brAngle = angle(right, bottom)
    blAngle = angle(bottom, left)
    
    
    print("angles")
    print(trAngle)
    print(tlAngle)
    print(brAngle)
    print(blAngle)
    
    # find a logic to check for offset points
    if (tlAngle-1.57 > 0.03  and blAngle-1.57<0.03 and brAngle-1.57<0.03):
        #change the point
        tl = [tl[0], tr[1]]
    #more adjustment needed
    if (trAngle-1.57 > 0.03 and blAngle-1.57<0.03 and brAngle-1.57<0.03):
        #change the point
        tr = [tr[0], tl[1]]
    if (brAngle-1.57 > 0.03 and tlAngle-1.57<0.03 and trAngle-1.57<0.03):
        #change the point
        br = [br[0], bl[1]]
    if (blAngle-1.57 > 0.03 and tlAngle-1.57<0.03 and trAngle-1.57<0.03):
        #change the point
        bl = [bl[0], br[1]]
    return tl, tr, bl, br

In [12]:
def cropFunc(filename,points):
    
    img = cv2.imread(filename)

    rows,cols,ch = img.shape

    tl, tr, bl, br = cornerIdentify(points)
    
    
    print(tl)
    print(tr)
    print(bl)
    print(br)
    
    distw = math.sqrt( (tl[0] - tr[0])**2 + (tl[1] - tr[1])**2 )
    
    distl = math.sqrt( (tl[0] - bl[0])**2 + (tl[1] - bl[1])**2 )
    
    print(distw)
    print(distl)
    
    
    # adjustment happen here 
    print("check")
    print(tl)
    tl, tr, bl, br = adjustmentfunc(tl, tr, bl, br)
    pts1 = np.float32([tl,tr,bl,br])
    
    print(tl)
    if(distw > distl):
        # this is landscape
        print("landscape")
        pts2 = np.float32([[0,0],[4000,0],[0,3000],[4000,3000]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        dst = cv2.warpPerspective(img, M, (4000,3000))
        cv2.imwrite("finalResult.jpg",dst)
        
        colorImage  = Image.open("finalResult.jpg")
        transposed  = colorImage.transpose(Image.ROTATE_90)
        transposed.save("finalResult.jpg")
        
        
    else:
        pts2 = np.float32([[0,0],[3000,0],[0,4000],[3000,4000]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        dst = cv2.warpPerspective(img, M, (3000,4000))
        cv2.imwrite("finalResult2.jpg",dst)
       
    
    
    return tl, tr, bl, br
    # try to recover some of the lost lines in the image
    #dst = cv2.erode(dst, None, iterations=2)


#     plt.subplot(121),plt.imshow(img),plt.title('Input')
#     plt.subplot(122),plt.imshow(dst),plt.title('Output')
    
#     plt.rcParams['figure.figsize'] = [200, 50]
#     fig=plt.figure(figsize=(180, 160))
    

## Corner Points selection

In [13]:
def findMinMaxPoints(points):
    
    
    xValue = []
    yValue = []
    for i in points:
        xValue.append(i[0])
        yValue.append(i[1])
    
    Data = {
        'x': xValue,
        'y': yValue
    }
    # convert into pandas
    df = pd.DataFrame(Data,columns=['x','y'])
    
    kmeans = KMeans(n_clusters =4).fit(df)
    centroids = kmeans.cluster_centers_
    #print(centroids)
    #print(kmeans.labels_)
    
    
    #figure what each index in cntroids represents which element out of the 4
    tlv, trv, blv, brv = cornerIdentify(centroids)
    for i, val in enumerate(centroids):
        if (val==tlv).all():
            tl = i
        elif (val==trv).all():
            tr = i
        elif (val==brv).all():
            br = i
        elif (val==blv).all():
            bl = i
    
   
    
    plabel = kmeans.labels_
    trl = []
    tll = []
    brl = []
    bll = []
    
    #separate all the points into different clusters 
    for i,val in enumerate(plabel):
        if val == tl:
            tll.append(points[i])
        elif val == tr:
            trl.append(points[i])
        elif val == br:
            brl.append(points[i])
        elif val == bl:
            bll.append(points[i])
            
            
    # find tl and br
    
    tlPoint = sorted(tll,key=sum)[0]
    brPoint = sorted(brl, key=sum)[-1]
    
    #find the tr and bl by checking the surrounding colors
    
    trPoint = ambCornerDetect(trl)
    blPoint = ambCornerDetect(bll)
    
    finalList = []
    finalList.append([tlPoint[0],tlPoint[1]])
    finalList.append([brPoint[0],brPoint[1]])
    finalList.append(trPoint)
    finalList.append(blPoint)
    
    return finalList


In [14]:
def ambCornerDetect(ls):
    im = Image.open('result.jpg')
    img = cv2.imread('result.jpg')
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    rgb_im = im.convert('RGB')
    
    #for all the points in each corner find the surrounding pixels color
    
    bestCorner = cPoints(-1,-1)
    
    for i in ls:
        
        newP = cPoints(i[0],i[1])
        prob = 0
        
        #check the surrounding
        
        xc = -2
        counter = 0
        while xc < 3:
            yc =-2
            while yc<3:
                counter = counter +1
                
                if (int(i[0]+xc)) < 0:
                    inX = 0
                else:
                    inX = int(i[0]+xc)
                              
                if (int(i[1]+yc)) < 0:
                    iny = 0
                else:
                    iny = int(i[1]+yc)
                #print(inX,iny)
                r, g, b = rgb_im.getpixel((inX, iny))
                sumT = r + g + b
                if sumT < 20:
                    prob = prob+1
                yc = yc +1
            xc = xc +1
        
        prob = prob / counter
        newP.assignP(prob)
        
        if prob > bestCorner.P:
            bestCorner = newP
        
        x,y = i.ravel()
        cv2.circle(img,(x,y),3,200,-1)
    return [bestCorner.x,bestCorner.y]

In [15]:
def mainProcessFun(filename):
    contourPoints = edgeDetectFun(filename)
    points = findMinMaxPoints(contourPoints)
    cropFunc(filename,points)

## Testing Functions

In [17]:
import time
start_time = time.time()
mainProcessFun('4.jpg')
print("--- %s seconds ---" % (time.time() - start_time))

3.2344094825902854
[115, 193]
[1552, 193]
[111, 2114]
[1557, 2040]
1437.0
1921.0041644931434
check
[115, 193]
angles
1.568714080975531
1.5680892408251421
1.5223723611685522
1.6240096242103612
[115, 193]
--- 0.9945988655090332 seconds ---
